In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils

import torchvision.models as models
import torchvision.transforms.v2 as v2

from PIL import Image
import os
import json

### Подготовка модели ResNet34 и замена последнего и предпосл слоев нейросети под наши цели

In [48]:
dataset_folder = "ActorsDataset"
device = "cuda" if torch.cuda.is_available() else "cpu"

resnet34_weights = models.ResNet34_Weights.DEFAULT
resnet34_transforms = resnet34_weights.transforms()

model = models.resnet34(weights=resnet34_weights).to(device)
model.fc = nn.Sequential(
    nn.Linear(512, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 7) # Корректируем посл слой под наши цели
)

for param in model.parameters(): # вся модель не будет обучаться
    param.requires_grad = False

#for param in model.layer4.parameters(): # оптимизируем только последний и предпосл слои
#    param.requires_grad = True

for param in model.fc.parameters():
    param.requires_grad = True

### Используйте файл actors_images_crawler.ipynb чтобы скачать изображения актеров и создать требуемую структуру
### Далее создаем объекты классов Dataset и Dataloader
### Оптимизатор и функция потерь

In [51]:
class ActorsDataset(utils.Dataset):
    def __init__(self, path: str, train: bool = True, transforms=None):
        self.path = os.path.join(path, "train" if train else "test")
        self.transforms = transforms

        with open("format.json", "r", encoding="utf-8") as fp:
            self.schema = json.load(fp)

        self.images = []
        self.len_images = 0

        for folder, target in self.schema.items():
            actor_path = os.path.join(self.path, folder)
            actor_images = os.listdir(actor_path)
            self.len_images += len(actor_images)
            self.images.extend(
                (os.path.join(actor_path, img), target) for img in actor_images
            )
    
    def __getitem__(self, item):
        path, target = self.images[item]
        
        image = Image.open(path).convert("RGB")
    
        if self.transforms:
            image = self.transforms(image).to(device)
    
        return image, target

    def __len__(self):
        return self.len_images


actors_train_dataset = ActorsDataset(dataset_folder, train=True, transforms=resnet34_transforms)
actors_train_dataloader = utils.DataLoader(actors_train_dataset, batch_size=16, shuffle=True)

test_dataset = ActorsDataset(dataset_folder, train=False, transforms=resnet34_transforms)
test_dataloader = utils.DataLoader(test_dataset, batch_size=32, shuffle=False)

optimizer = optim.Adam(
    #list(model.fc.parameters()) + list(model.layer4.parameters()),
    model.fc.parameters(),
    lr=0.001
    #weight_decay=0.0001
)
loss_func = nn.CrossEntropyLoss()


### Обучение

In [54]:
#best_result = float("inf")

epochs = 10
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for x_train, y_train in actors_train_dataloader:
        optimizer.zero_grad()
        predict = model(x_train)
        loss = loss_func(predict, y_train)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(actors_train_dataloader)

    # if avg_loss < best_result: # сохраняем параметры модели и оптимизатора если рещультаты стали лучше
    #     best_result = avg_loss
    #     torch.save({
    #         'epoch': epoch + 1,
    #         'model_state_dict': model.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': avg_loss
    #     }, f"{epoch + 1}_result.tar")

    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

Epoch 1/10, Loss: 1.1172
Epoch 2/10, Loss: 0.5057
Epoch 3/10, Loss: 0.3338
Epoch 4/10, Loss: 0.2321
Epoch 5/10, Loss: 0.1570
Epoch 6/10, Loss: 0.1466
Epoch 7/10, Loss: 0.1253
Epoch 8/10, Loss: 0.1058
Epoch 9/10, Loss: 0.1023
Epoch 10/10, Loss: 0.0914


### Тестирование

In [57]:
model.eval()
total = 0
correct = 0
test_loss = 0

with torch.no_grad():
    for x_test, y_test in test_dataloader:
        predict = model(x_test)
        loss = loss_func(predict, y_test)
        test_loss += loss.item()
        preds = torch.argmax(predict, dim=1)
        correct += (preds == y_test).sum().item()
        total += y_test.size(0)

print(f"\nTest Loss: {test_loss / len(test_dataloader):.4f}")
print(f"Test Accuracy: {correct / total:.4f}")



Test Loss: 1.2716
Test Accuracy: 0.6511
